In [29]:
from keras.engine import Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface import utils
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import backend as K
from sklearn.utils import class_weight
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read in ratings
ratings = np.genfromtxt('./Projekt_SGE_Assessment_ErikK.txt')

In [6]:
# Round (obviously imbalanced sets)
ratings = np.round(ratings, 0)
print(np.unique(ratings, return_counts=True))

(array([0., 1., 2., 3., 4., 5.]), array([197, 128,  75,  70,  50,  48], dtype=int64))


In [10]:
### Distribute images across folders
for r in np.unique(ratings).astype(np.int8):
    os.makedirs('./classification/{}-stars'.format(r))

In [19]:
### Move all images according to ratings
images = sorted(os.listdir('./all_females/'))

### Make sure lengths of ratings and images correspond 
assert len(ratings) == len(images)

### Loop throgh all images and store them in the dedicated folders
for i, r in enumerate(ratings.astype(np.int8)):
    src = os.path.join('./all_females', images[i])
    dest = os.path.join('./classification/{}-stars'.format(r), images[i])
    shutil.copy(src, dest)

## Time to use data augmentation with streaming

In [25]:
datagen = ImageDataGenerator(rescale=1./255, 
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest',
                             preprocessing_function=utils.preprocess_input)

In [26]:
train_generator = datagen.flow_from_directory(
    './classification/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 568 images belonging to 6 classes.


## Some random model

In [27]:
# Add custom layers
vgg_base = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='max')
last_layer = vgg_base.get_layer('global_max_pooling2d_1').output
X = Dropout(0.2)(last_layer)
X = Dense(256, activation='relu', name='fc6')(X)
X = Dropout(0.2)(X)
X = Dense(128, activation='relu', name='fc7')(X)
X = Dropout(0.2)(X)
output = Dense(len(np.unique(ratings)), activation='softmax')(X)
model = Model(inputs=vgg_base.input, outputs=output)

In [28]:
# Freeze vgg layers
for layer in vgg_base.layers:
    layer.trainable = False

In [30]:
# Compute class weights since we have unbalaned classes
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(ratings), 
                                                 ratings)

In [32]:
model.compile(optimizer=Adam(clipnorm=1.0),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# P.S. Need to store some validation images before fitting
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=1,
    class_weight=class_weight
)